In [178]:
import pandas as pd
import numpy as np

El encoder por defecto no puede decodificar algún caracter raro así que usuaré otro encoder

In [179]:
df = pd.read_csv('recieved_dirty_dataset.csv', encoding='latin-1')

In [180]:
df.head()

,ID_REGISTRO_X,date_,price,bedrooms,BATH,sqft_living,yr_built,condition
0,7527200030,20141229T000000,700000.0,5.0,2.50,2830.0,1979,5
1,3300701615,20140930T000000,655000.0,50.0,2.50,NaN,2002,3
2,9528102996,20141207T000000,549000.0,3.0,1.75,1540.0,2014,3
3,2008200060,20140624T000000,160000.0,3.0,1.50,1010.0,1962,Fouur
4,3394100030,20140909T000000,975000.0,4.0,2.50,2720.0,1989,3


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_REGISTRO_X  600 non-null    int64  
 1   date_          600 non-null    object 
 2   price          569 non-null    object 
 3   bedrooms       597 non-null    float64
 4   BATH           595 non-null    float64
 5   sqft_living    537 non-null    float64
 6   yr_built       600 non-null    object 
 7   condition      600 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 37.6+ KB


En primer lugar me fijo en que las columnas tienen nombres no estandarizados, así que es lo primero que voy a corregir

In [182]:
df = df.rename(columns={
    'ID_REGISTRO_X': 'id',
    'date_': 'date',
    'BATH': 'bathrooms'
})

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,yr_built,condition
0,7527200030,20141229T000000,700000.0,5.0,2.50,2830.0,1979,5
1,3300701615,20140930T000000,655000.0,50.0,2.50,NaN,2002,3
2,9528102996,20141207T000000,549000.0,3.0,1.75,1540.0,2014,3
3,2008200060,20140624T000000,160000.0,3.0,1.50,1010.0,1962,Fouur
4,3394100030,20140909T000000,975000.0,4.0,2.50,2720.0,1989,3


Lo siguiente que se me ocurre es revisar si hay duplicados y eliminarlos

In [183]:
duplicados = df.duplicated().sum()
print(f'{duplicados} duplicados y {df.shape[0]} filas en total')

51 duplicados y 600 filas en total


In [184]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
print(f'{df.shape[0]} filas en total')

549 filas en total


## COLUMNA ID

Parece que la columna id se salva, vamos a checkear por si acaso que no tenga nulos antes de pasar a la siguiente

In [185]:
df['id'].isna().sum()

np.int64(0)

Vemos que está bien así que vamos con la columna date

## COLUMNA DATE

In [186]:
df['date'].head()

0    20141229T000000
1    20140930T000000
2    20141207T000000
3    20140624T000000
4    20140909T000000
Name: date, dtype: object

Los primeros registros vemos que siguen un patrón de formato determinado, voy a recogerlo en una expresión reguular y comprobar si hay valores extraños

In [187]:
df['date'] = df['date'].astype(str)

df['date_temporal'] = pd.to_datetime(
    df['date'].str.replace('T000000', ''),
    format='%Y%m%d',
    errors='coerce' # Convierte valores no válidos a NaT
)

# Filtrar las filas donde la conversión falló (es decir, donde el valor es NaT)
anomalous_dates = df[df['date_temporal'].isnull()]


anomalous_counts = anomalous_dates['date'].value_counts()
print(anomalous_counts)   

del df['date_temporal']

date
ERROR-Año-Ñ0000    6
Name: count, dtype: int64


In [188]:
df['date'] = df['date'].replace('ERROR-Año-Ñ0000', np.nan)
df['date'] = df['date'].str.replace('T000000', '', regex=False)

df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='coerce')

rows_before = len(df)
df.dropna(subset=['date'], inplace=True)
rows_after = len(df)

print(f"Filas eliminadas debido a NaT: {rows_before - rows_after}")
print(f"Nulos (NaT) finales en 'date': {df['date'].isnull().sum()}")

Filas eliminadas debido a NaT: 6
Nulos (NaT) finales en 'date': 0
